In [7]:
# Required install libraries:
#        datasets
#        transformers
#        PyTorch
#        
import os

In [8]:
# Loads the data from data_XXXX.txt and label_XXXX.txt files
def load_local_dataset(data_dir):
    texts = []
    labels = []
    
    # Iterate over the files in your dataset directory
    for filename in os.listdir(data_dir):
        if filename.startswith("data_") and filename.endswith(".txt"):
            data_filepath = os.path.join(data_dir, filename)
            label_filepath = data_filepath.replace("data_", "label_")  # Assuming matching filenames for data and labels
            
            # Read the text
            with open(data_filepath, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                texts.append(text)
            
            # Read the corresponding label
            with open(label_filepath, 'r', encoding='utf-8') as f:
                label = f.read().strip() 
                labels.append(label)
    
    # Return a list of dicts with 'text' and 'label' keys
    return [{"text": t, "label": l} for t, l in zip(texts, labels)]

# Define the path to your dataset folder
data_dir = "./training_data"
dataset = load_local_dataset(data_dir)

In [9]:
# Loads data from data_XXXX.txt and label_XXXX.txt files
def load_local_dataset(data_dir):
    texts = []
    labels = []
    
    # Iterate over the files in your dataset directory
    for filename in os.listdir(data_dir):
        if filename.startswith("data_") and filename.endswith(".txt"):
            data_filepath = os.path.join(data_dir, filename)
            label_filepath = data_filepath.replace("data_", "label_")  # Assuming matching filenames for data and labels
            
            # Read the text
            with open(data_filepath, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                texts.append(text)
            
            # Read the corresponding label
            with open(label_filepath, 'r', encoding='utf-8') as f:
                label = f.read().strip() 
                labels.append(label)
    
    # Return a list of dicts with 'text' and 'label' keys
    return [{"text": t, "label": l} for t, l in zip(texts, labels)]

# Define the path to your dataset folder
data_dir = "./training_data"
dataset = load_local_dataset(data_dir)

In [10]:
# Convert loaded data to Hugging Face Dataset format
from datasets import Dataset

# Load the dataset into a Hugging Face Dataset object
dataset = Dataset.from_dict({"text": [item['text'] for item in dataset],
                             "label": [item['label'] for item in dataset]})

# Optionally split into train/validation/test sets
dataset = dataset.train_test_split(test_size=0.2)  # 80/20 train-test split

In [11]:
# Tokenization and Fine-Tuning
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModel

# Load the tokenizer
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load pre-trained BERT model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels to your task

def str2int(str):
    if str == "nao-protesto":
        return 0
    return 1

# Tokenize the dataset
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    tokenized["label"] = [str2int(label) for label in examples["label"]]
    return tokenized

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [12]:
# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Start training
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.7274514436721802, 'eval_runtime': 0.0571, 'eval_samples_per_second': 17.5, 'eval_steps_per_second': 17.5, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.6123903393745422, 'eval_runtime': 0.0542, 'eval_samples_per_second': 18.465, 'eval_steps_per_second': 18.465, 'epoch': 2.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.5765517354011536, 'eval_runtime': 0.1145, 'eval_samples_per_second': 8.732, 'eval_steps_per_second': 8.732, 'epoch': 3.0}
{'train_runtime': 12.3468, 'train_samples_per_second': 0.972, 'train_steps_per_second': 0.243, 'train_loss': 0.633892297744751, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=0.633892297744751, metrics={'train_runtime': 12.3468, 'train_samples_per_second': 0.972, 'train_steps_per_second': 0.243, 'total_flos': 3157332664320.0, 'train_loss': 0.633892297744751, 'epoch': 3.0})

In [13]:
# Save both the model and tokenizer
trainer.save_model("./trained_model")  # Saves the model
tokenizer.save_pretrained("./trained_model")  # Saves the tokenizer

('./trained_model\\tokenizer_config.json',
 './trained_model\\special_tokens_map.json',
 './trained_model\\vocab.txt',
 './trained_model\\added_tokens.json',
 './trained_model\\tokenizer.json')

# Now let's run inference on the trained model

In [14]:
# Path to your saved model
model_path = "./trained_model"

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def getClass(pred):
    if pred == 0:
        return 'nao-protesto'
    return 'protesto'

In [17]:
exampleText = ""

# Tokenize the input text
inputs = tokenizer(exampleText, return_tensors="pt", truncation=True, padding=True, max_length=512)


In [18]:
import torch
import torch.nn.functional as F

# Run inference with the trained model
with torch.no_grad():
    outputs = model(**inputs)

# Get the model's logits and apply softmax to get probabilities
logits = outputs.logits
probabilities = F.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probabilities, dim=-1).item()

# Output the result
print(f"Predicted class: {getClass(predicted_class)}")
print(f"Probabilities: {probabilities}")

Predicted class: nao-protesto
Probabilities: tensor([[0.5058, 0.4942]])
